In [2]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import glob

from mpl_toolkits.axes_grid1 import make_axes_locatable
import time

import preseason.tools as sf
import preseason.onset_demise as od
import preseason.plotting as pp
from scipy import stats
import dask.array as da

import cartopy
import cartopy.io.shapereader as shapereader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import rioxarray
import geopandas as gpd

In [3]:
### Allows us to use dask to speed up some calculations ###
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=4, memory_limit='8GB', threads_per_worker=4)
client = Client(cluster)

In [4]:


PERU= {'lon': slice(-85+365,-15+365), 'lat': slice(0,-70)}


In [5]:
### Onset Demise for Precipitation ###

# Changed chunking strategy. Chunking the entire time dimension causes shapefile clipping to fail #

precip_data_dir = '/data/deluge/reanalysis/REANALYSIS/ERA5/2D/daily/precip/'

precip_files = glob.glob(precip_data_dir+'precip.[12]*')


ds_p = xr.open_mfdataset(precip_files, parallel=True)


precip_data = ds_p['precip'].sel(latitude = PERU['lat'], longitude = PERU['lon'])

precip_anom = precip_data - precip_data.mean(dim='time')

#precip_anom = precip_anom

In [ ]:
lon_name = 'longitude'  # whatever name is in the data

# Adjust lon values to make sure they are within (-180, 180)
precip_anom['_longitude_adjusted'] = xr.where(
    precip_anom[lon_name] > 180,
    precip_anom[lon_name] - 360,
    precip_anom[lon_name])

# reassign the new coords to as the main lon coords
# and sort DataArray using new coordinate values
ds = (precip_anom.swap_dims({lon_name: '_longitude_adjusted'}).sel(**{'_longitude_adjusted': sorted(precip_anom._longitude_adjusted)}).drop(lon_name))

ds = ds.rename({'_longitude_adjusted': lon_name})
precip_anom = ds.transpose('time','latitude','longitude')

precip_anom.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)
precip_anom.rio.write_crs("epsg:4326", inplace=True)

In [8]:

fname = '/home/gbromley/peru_basins/peru_basins_lv5.shp'

Peru_db = gpd.read_file(fname)


In [ ]:
results = []
for i in np.arange(len(Peru_db)):
    
    basin_id = Peru_db.loc[i,'HYBAS_ID']
    single_basin = gpd.GeoSeries(Peru_db.loc[i,'geometry'])
    

    clipped = precip_anom.rio.clip(single_basin, precip_anom.rio.crs, drop=False)
    basin_ts = clipped.mean(dim=['latitude','longitude'])
    results.append(basin_ts)

    

In [16]:
combined_data = xr.concat(results, dim='HYBAS_ID')
#ds = combined_data.to_xarray()
#ds = ds.rename({'STATION': 'ghcnd_id', 'DATE': 'time'})

In [20]:
combined_data = combined_data.assign_coords({'HYBAS_ID':Peru_db.loc[:,'HYBAS_ID'].values})

In [ ]:
combined_data.to_netcdf('Peru_allBasins_precip.nc')

In [ ]:
combined_data.isel(HYBAS_ID=3).plot()

distributed.core - ERROR - 'tcp://127.0.0.1:41095'
Traceback (most recent call last):
  File "/home/gbromley/anaconda3/envs/obswetdryseason/lib/python3.7/site-packages/distributed/core.py", line 575, in handle_stream
    handler(**merge(extra, msg))
  File "/home/gbromley/anaconda3/envs/obswetdryseason/lib/python3.7/site-packages/distributed/scheduler.py", line 5342, in handle_release_data
    ws: WorkerState = parent._workers_dv[worker]
KeyError: 'tcp://127.0.0.1:41095'
distributed.utils - ERROR - 'tcp://127.0.0.1:41095'
Traceback (most recent call last):
  File "/home/gbromley/anaconda3/envs/obswetdryseason/lib/python3.7/site-packages/distributed/utils.py", line 638, in log_errors
    yield
  File "/home/gbromley/anaconda3/envs/obswetdryseason/lib/python3.7/site-packages/distributed/scheduler.py", line 4327, in add_worker
    await self.handle_worker(comm=comm, worker=address)
  File "/home/gbromley/anaconda3/envs/obswetdryseason/lib/python3.7/site-packages/distributed/scheduler.py",

KeyboardInterrupt: 

distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
